# Regression

## Plan
**Wie sollten wir unser Budget auf unsere Marketingkanäle verteilen?**

Die Marketingabteilung möchte wissen, wie sie den resultierenden Umsatz und somit den Erfolg einer Marketingaktivität voraussagen können. Dies soll einerseits dabei helfen, dass Budget zu begründen und optimal auf verschiedene Kanäle zu verteilen. Gleichzeitig ist es aber auch eine Möglichkeit den Kampagnenerfolg zu bewerten, indem man prüft, inwieweit der tatsächliche Umsatz über bzw. unter dem prognostizierten Wert liegt.

![](../bmc_recommender.pdf)

## Data
Die Basis der Regressionsanalyse bildet ein Datensatz, in dem das Budget und der Umsatz aller vergangenen Kampagnen eingetragen sind. Für jede Kampagne gibt es die Information, wie viel Geld pro Kanal ausgegeben wurde und wie hoch der generierte Umsatz waren. Der originale Datensatz stammt von [kaggle](https://www.kaggle.com/harrimansaragih/dummy-advertising-and-sales-data).

Da es sich bei dem Original-Datensatz um einen Trainingsdatensatz handelt, besteht er nur aus wenigen Variablen *(TV, Print, Social Media und Influencer)*. Zudem besteht zwischen der Variable TV und der Zielvariable Sales eine 1:1 Korrelation, welche in der Realität unwahrscheinlich ist. Da die Umsetzung der Use Cases jedoch nur exemplarisch durchgeführt wird und ich keinen anderen passenden Datensatz zum Thema gefunden habe, entschied ich mich bei diesem zu bleiben und die Spalte *TV* aus der Modellierung auszuschließen. 

Der vollständige Code befindet sich [hier](Regression_Marketing.ipynb).

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../Regression_marketing_data.csv', delimiter=',', encoding = 'unicode_escape')
df.head()

,TV,Print,Social,Influencer,Sales
0,16.0,6.566231,2.907983,Mega,54.732757
1,13.0,9.237765,2.409567,Mega,46.677897
2,41.0,15.886446,2.913410,Mega,150.177829
3,43.0,30.020028,6.922304,Mega,298.246340
4,15.0,8.437408,1.405998,Micro,56.594181


In [3]:
df.describe()

,TV,Print,Social,Sales
count,4545.000000,4568.000000,4566.000000,4566.000000
mean,53.874587,18.160356,3.323956,192.466602
std,26.116303,9.676958,2.212670,93.133092
min,1.000000,0.000684,0.000031,31.199409
25%,32.000000,10.525957,1.527849,112.322882
50%,53.000000,17.859513,3.055565,189.231172
75%,76.000000,25.649730,4.807558,272.507922
max,100.000000,48.871161,13.981662,364.079751


## Modell
Nach dem Data-Splitting, führe ich zunächst eine explorative Datenanalyse durch. Zudem habe ich eine weitere Variable *Total Sum* eingefügt, die die Gesamtausgaben pro Kampagne angab. Die Plots zeigen eine positive, lineare Korrelation zwischen den Features und unserer Zielvariablen. 

Anschließend erstelle ich eine Pipeline, die verschiedene Pre-Processing Schritte mit der Modellierung verband. Aufgrund der linearen Korrelation entscheide ich mich zunächst für ein **simples lineares Regressionsmodell** mit dem Feature *Total Sum*. Dieses gibt jedoch keinen Aufschluss darüber, wie das Budget auf verschiedene Marketingkanäle verteilt werden sollte. Deswegen entscheide ich mich trotz des bereits guten Scores für ein weiteres **multiples Regressionsmodell** basierend auf den anderen Features. Dieses erzielt einen vergleichbaren Score und berücksichtigt zudem mehrere Kanäle. Durch den Einsatz von Hyperparameter Tuning kann das Modell jedoch nicht mehr verbessert werden, was bedeutet, dass das Modell mit den Standardwerten bereits das beste Ergebnis erzielt.

Um auch neuere Algorithmen auf den Datensatz anzuwenden, arbeite ich zusätzlich noch mit einer **Lasso Regression** und XGBoost. Da die Plots bereits eine deutliche Linearität der Korrelation zeigten, bringt die Lasso Regression jedoch keine weitere Verbesserung des Modells. 

Mit dem **XGBoost Algorithmus** kann der Score im Vergleich zu den anderen Modellen nochmal gesteigert werden, weswegen ich mich am Ende auch für dieses Modell entscheide. Der XGBoost Algorithmus kombiniert mehrere schwächere und einfache Modelle miteinander, um Vorhersagen zu treffen. Es werden so lange weitere Modelle hinzugefügt, bis sich die Vorhersage nicht mehr verbessert. Des Weiteren bietet XGBoost die Möglichkeit zu sehen, wie stark einzelne Features in das Modell eingehen. Diese Feature Importance hat gezeigt, dass insbesondere die Ausgaben für Printwerbung einen großen Einfluss auf die Vorhersage hat. Dies ist wenig überraschend, da dieses Feature auch die stärkste Korrelation mit unserer Zielvariable Sales aufzeigt.

Um das Modell abschließend zu bewerten, wird es auf den **Testdatensatz** angewendet. Dort erzielt es einen vergleichbaren Score wie in der Testgruppe. Deswegen können wir davon ausgehen, dass das entwickelte Modell weder over- noch underfitted ist. Es erklärt somit 75-78% der Abweichung der Werte.

## Evaluation
Von der Vorhersage des Absatz profitieren auch weitere Abteilungen. Zum Beispiel kann damit auch der Website Traffic für den Zeitraum von Kampagnen vorhergesagt werden, um die nötigen technischen Voraussetzungen zu treffen. Außerdem kann der Einkauf damit den Bedarf der beworbenen Produkte besser schätzen.

Wichtig bei der Interpretation ist jedoch, dass es sich bei dem Datensatz nach wie vor um Testdaten handelt. Die Linearität zwischen den Variablen wird in der Realität so vermutlich nicht vorkommen. Außerdem beinhaltet der Datensatz nur sehr wenige Kanäle.